# Memoria del Proyecto – Student Performance (EDA)

**Autor:** _Tu nombre aquí_  
**Fecha:** _Actualiza la fecha_

## Introducción
Analizamos un dataset de rendimiento académico para contrastar 4 hipótesis:
1. Un mayor nivel educativo de los padres se asocia con mejores calificaciones.
2. Haber completado el curso de preparación para el examen mejora las notas.
3. Diferencias por género: alumnas mejores en lectura/escritura; alumnos en matemáticas.
4. Diferencias por grupos `race/ethnicity` en los resultados académicos.

## Fuentes de datos
- `src/data/Student_performance_data_.csv` (dataset subido por el alumno).


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from utils.eda_utils import encode_parental_education_ordinal, ttest_by_binary, ttest_two_groups, anova_by_group

pd.set_option("display.max_columns", None)
%matplotlib inline

DATA_PATH = "src/data/Student_performance_data_.csv"
df = pd.read_csv(DATA_PATH)
df.head()


## Diccionario de datos (estimado)

In [ ]:

df.info()
display(df.describe(include='all'))


## Limpieza de datos

In [ ]:

# Estandarización de nombres de columnas a snake_case
df.columns = [c.strip().lower().replace(' ', '_').replace('/', '_') for c in df.columns]

# Chequeos básicos de calidad
missing = df.isna().mean().sort_values(ascending=False)
duplicates = df.duplicated().sum()
print("Duplicados:", duplicates)
missing.head(10)


## Exploratorio general

In [ ]:

numeric = df.select_dtypes(include=['int64','float64']).columns.tolist()
cat = [c for c in df.columns if c not in numeric]

display(df[numeric].describe())
for c in cat:
    print("\n", c)
    display(df[c].value_counts())


## Hipótesis 1 – Educación de los padres y calificaciones

In [ ]:

if "parental_level_of_education" in df.columns:
    df["parental_level_of_education"] = df["parental_level_of_education"].str.lower()
    df["parental_edu_ord"] = encode_parental_education_ordinal(df["parental_level_of_education"])

    for score in ["math_score", "reading_score", "writing_score"]:
        if score in df.columns:
            plt.figure()
            sns.boxplot(x="parental_level_of_education", y=score, data=df, order=[
                "some high school","high school","some college","associate's degree","bachelor's degree","master's degree"
            ])
            plt.xticks(rotation=30, ha='right'); plt.title(f"{score} por nivel educativo de los padres")
            plt.show()
            print("ANOVA:", anova_by_group(df.dropna(subset=["parental_level_of_education", score]), score, "parental_level_of_education"))
else:
    print("No se encontró la columna 'parental_level_of_education'. Ajusta los nombres según tu dataset.")


## Hipótesis 2 – Curso de preparación y calificaciones

In [ ]:

if "test_preparation_course" in df.columns:
    for score in ["math_score", "reading_score", "writing_score"]:
        if score in df.columns:
            plt.figure()
            sns.boxplot(x="test_preparation_course", y=score, data=df)
            plt.title(f"{score} vs test_preparation_course")
            plt.show()
            res = ttest_by_binary(df, score, "test_preparation_course", "completed")
            print(res)
else:
    print("No se encontró la columna 'test_preparation_course'.")


## Hipótesis 3 – Diferencias por género

In [ ]:

if "gender" in df.columns:
    for score in ["math_score", "reading_score", "writing_score"]:
        if score in df.columns:
            plt.figure()
            sns.boxplot(x="gender", y=score, data=df)
            plt.title(f"{score} por género")
            plt.show()
            cats = df["gender"].dropna().unique().tolist()
            if len(cats) >= 2:
                a, b = cats[0], cats[1]
                print(ttest_two_groups(df, score, "gender", a, b))
else:
    print("No se encontró la columna 'gender'.")


## Hipótesis 4 – Diferencias por race/ethnicity

In [ ]:

if "race_ethnicity" in df.columns:
    for score in ["math_score", "reading_score", "writing_score"]:
        if score in df.columns:
            plt.figure()
            sns.boxplot(x="race_ethnicity", y=score, data=df)
            plt.title(f"{score} por race/ethnicity")
            plt.show()
            print("ANOVA:", anova_by_group(df.dropna(subset=["race_ethnicity", score]), score, "race_ethnicity"))
else:
    print("No se encontró la columna 'race_ethnicity'.")


## Conclusiones
- Resume si cada hipótesis se **acepta** o **se rechaza** (p-values y tamaños de efecto).
- Limitaciones del dataset y sesgos potenciales.
- Próximos pasos: variables adicionales, modelos, análisis causal.
